# Требования к отчету:
## Отчет по лабораторной работе должен содержать:

# титульный лист;
## описание задания;
## текст программы;
## экранные формы с примерами выполнения программы.

In [552]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from IPython.display import Image
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from surprise import SVD, Dataset, Reader
from surprise.model_selection import PredefinedKFold
from collections import defaultdict
from surprise.accuracy import rmse
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline 
sns.set(style="ticks")

# Считывание входных данных

In [241]:
data = pd.read_csv('/Users/wanhao/Downloads/毛子2/ммо/wiki_movie_plots_deduped.csv',encoding ='gb18030')
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,point
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",76
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",93
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",80
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,84
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,79


In [244]:
data.shape


(34811, 9)

In [245]:
data.drop_duplicates(['Title'])

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,point
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",76
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",93
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",80
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,84
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,79
...,...,...,...,...,...,...,...,...,...
34806,2017,Midnight Runners,South_Korean,Jason Kim (Kim Ju-hwan),"Park Seo-joon, Kang Ha-neul",unknown,https://en.wikipedia.org/wiki/Midnight_Runners,The story is about two Korean National Police ...,NaN
34807,2017,The Mimic,South_Korean,Huh Jung,"Yum Jung-ah, Park Hyuk-kwon",unknown,https://en.wikipedia.org/wiki/The_Mimic_(film),A woman whose son is missing takes in a young ...,NaN
34808,2017,V.I.P,South_Korean,Park Hoon-jung,"Jang Dong-gun, Kim Myung-min, Lee Jong-suk, Pa...",unknown,https://en.wikipedia.org/wiki/V.I.P._(2017_film),The son of a high-ranking North Korean officia...,NaN
34809,2017,The Table,South_Korean,Kim Jong-kwan,"Im Soo-jung, Jung Yu-mil, Han Ye-ri, Jung Eun-...",unknown,https://en.wikipedia.org/wiki/The_Table_(2016_...,The film follows four conversations between tw...,NaN


In [246]:
overview_data_=data[data['Title'].notnull()]

In [247]:
Plot_data=overview_data_[overview_data_['Plot'].notnull()]

In [556]:
title=Plot_data['Title'].values

In [557]:
title[0:5]

array(['Kansas Saloon Smashers', 'Love by the Light of the Moon',
       'The Martyred Presidents', 'Terrible Teddy, the Grizzly King',
       'Jack and the Beanstalk'], dtype=object)

In [250]:
Plot=Plot_data['Plot'].values
Plot[0:5]

"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"

In [251]:
index=np.arange(len(Plot))
index

array([    0,     1,     2, ..., 34802, 34803, 34804])

In [252]:
%%time
tfidf = TfidfVectorizer()
Plot_matrix = tfidf.fit_transform(Plot)
Plot_matrix

CPU times: user 6.55 s, sys: 163 ms, total: 6.71 s
Wall time: 6.71 s


<34805x141611 sparse matrix of type '<class 'numpy.float64'>'
	with 6392974 stored elements in Compressed Sparse Row format>

In [253]:
Plot_matrix

<34805x141611 sparse matrix of type '<class 'numpy.float64'>'
	with 6392974 stored elements in Compressed Sparse Row format>

# Фильтрация на основе содержания. Метод k-ближайших соседей


In [254]:
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_ids, X_title, X_overview):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_ids - массив идентификаторов объектов
        X_title - массив названий объектов
        X_overview - массив описаний объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'id': pd.Series(X_ids, dtype='int'),
            'title': pd.Series(X_title, dtype='str'),
            'overview': pd.Series(X_overview, dtype='str'),
            'dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

In [255]:
demo_id=1100
print(Plot[demo_id])

Shirley Mortimer (Ann Harding) is the second daughter of a wealthy London family, who view her as plain and treat her as little more than a servant. When her father's friend David Trent (Leslie Howard) visits, she becomes smitten. Upon hearing that he and his son are in need of a new domestic, she disguises herself as an elderly matron, Mrs. Hastings, and begins to work for him. He is a defense attorney, currently defending a man, painter Norman Harrington (Robert Williams), on the charge of murdering his wife.
As Mrs. Hastings, Shirley wins the friendship of David's son and dotes on David, making sure he takes care of himself. David, for his part, begins to suspect that "Mrs. Hastings" is not who she claims. Harrington is acquitted, and upon meeting Mrs. Hastings asks to paint her portrait; while doing so, he realizes she is actually a young woman, but agrees to keep her secret.
After spending an evening with Shirley (out of disguise) and her father, David realizes who "Mrs. Hastings"

In [256]:
demo_matrix = Plot_matrix[demo_id]

In [257]:
skr1 = SimpleKNNRecommender(Plot_matrix,index,title,Plot)

In [258]:
rec1 = skr1.recommend_for_single_object(15, demo_matrix)
rec1

,id,title,overview,dist
1604,1604,Stage Mother,Four years after her vaudevillian husband's de...,382611.142706
8472,8472,"Me, Natalie","From childhood, Brooklyn teenager, Natalie Mil...",353143.011871
1430,1430,Two Seconds,"As John Allen (Edward G. Robinson), a condemne...",318519.497918
16464,16464,C.O.G.,"David (Jonathan Groff) travels by bus, endurin...",315582.780870
17661,17661,Shine,A man (Geoffrey Rush) wanders through a heavy ...,313001.245529
15747,15747,The Stepfather,The film opens as Grady Edwards (Dylan Walsh) ...,300344.414168
5601,5601,Cattle Town,Mike McGann is sent by a government official i...,290106.270176
17401,17401,Shirley Thompson vs. the Aliens,"In 1950s Sydney, Shirley and her gang discover...",289499.994840
12623,12623,Big Bully,"Growing up in Hastings, Minnesota, young David...",289318.074927
1647,1647,Bright Eyes,5 year-old Shirley Blake (Shirley Temple) and ...,288528.109840


In [259]:
rec2 = skr1.recommend_for_single_object(15,demo_matrix, cos_flag = False)
rec2

,id,title,overview,dist
1604,1604,Stage Mother,Four years after her vaudevillian husband's de...,1.111206e+06
8472,8472,"Me, Natalie","From childhood, Brooklyn teenager, Natalie Mil...",1.137415e+06
1430,1430,Two Seconds,"As John Allen (Edward G. Robinson), a condemne...",1.167459e+06
16464,16464,C.O.G.,"David (Jonathan Groff) travels by bus, endurin...",1.169972e+06
17661,17661,Shine,A man (Geoffrey Rush) wanders through a heavy ...,1.172176e+06
15747,15747,The Stepfather,The film opens as Grady Edwards (Dylan Walsh) ...,1.182925e+06
5601,5601,Cattle Town,Mike McGann is sent by a government official i...,1.191548e+06
17401,17401,Shirley Thompson vs. the Aliens,"In 1950s Sydney, Shirley and her gang discover...",1.192057e+06
12623,12623,Big Bully,"Growing up in Hastings, Minnesota, young David...",1.192210e+06
1647,1647,Bright Eyes,5 year-old Shirley Blake (Shirley Temple) and ...,1.192872e+06


In [260]:
rec3 = skr1.recommend_for_single_object(15,demo_matrix,cos_flag = False, manh_flag = True)
rec3

,id,title,overview,dist
7429,7429,Walk Like a Dragon,Film's introduction:,9.623024e+06
33377,33377,A Saloon Wet with Beautiful Women,An erotic romance.[4],9.723718e+06
7606,7606,Heaven and Earth Magic,Smith explains:,9.731025e+06
33836,33836,Galaxy Turnpike,The film is set in 2265.[3],9.733403e+06
24876,24876,Chhalia,Chhalia is a family thriller.,9.763104e+06
22731,22731,Blind Massage,The film is set in Nanjing.[1],9.824532e+06
18965,18965,Death Is a Number,A man is persecuted by the number 9.,9.904456e+06
25161,25161,Laparwah,Laparwah is an out-and-out action film.,9.918407e+06
27623,27623,Apaaratha,Apaaratha is an emotional family film.,9.918978e+06
29554,29554,Achchani,Achchani is an emotional family film.,9.918978e+06


# Коллаборативная фильтрация. Метод на основе сингулярного разложения

In [359]:
data2=data.iloc[:15000,:]

In [360]:
len(data2['Director'].unique())

3956

In [361]:
len(data2['Title'].unique())

14324

In [362]:
def create_utility_matrix(data):
    itemField = 'Director'
    userField = 'Title'
    valueField = 'point'  
    
    userList = data[userField].tolist()
    itemList = data[itemField].tolist()
    valueList = data[valueField].tolist()    
    
    users = list(set(userList))
    items = list(set(itemList))    
    
    users_index = {users[i]: i for i in range(len(users))}    
    pd_dict = {item: [0.0 for i in range(len(users))] for item in items}    
    
    for i in range(0,data.shape[0]):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]    
        pd_dict[item][users_index[user]] = value    
    
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    
    return X, users_index, items_index


In [363]:
%%time
user_item_matrix, users_index, items_index = create_utility_matrix(data2)

CPU times: user 5.06 s, sys: 190 ms, total: 5.25 s
Wall time: 5.25 s


In [364]:
user_item_matrix

,Michael Cristofer,Mike Nichols,Chazz Palminteri,Norberto Barba,Eric Schaeffer,Steve Oedekerk,Charles Herman-Wurmfeld,John Wayne,2 directors,Thomas McGuane,...,Tom Moore,Blaine Novak,Chuck Sheetz,Glenn Tryon,Ken Weiderhorn,Frank Whaley,Jean de Limur,Roland D. Reed,Tod Browning,William Corrigan
Mackintosh and T.J.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
On Dress Parade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Scarlet Pages,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Last Run,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Smarty Cat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
One More Saturday Night,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Eddie Macon's Run,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Plain Clothes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Best in Show,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [365]:
user_item_matrix__test = user_item_matrix.loc[['The Martyred Presidents']]
user_item_matrix__test

,Michael Cristofer,Mike Nichols,Chazz Palminteri,Norberto Barba,Eric Schaeffer,Steve Oedekerk,Charles Herman-Wurmfeld,John Wayne,2 directors,Thomas McGuane,...,Tom Moore,Blaine Novak,Chuck Sheetz,Glenn Tryon,Ken Weiderhorn,Frank Whaley,Jean de Limur,Roland D. Reed,Tod Browning,William Corrigan
The Martyred Presidents,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [366]:
Title = np.delete(data2['Title'].unique(), 0)
Title = np.delete(Title, 1)
Title

array(['Love by the Light of the Moon',
       'Terrible Teddy, the Grizzly King', 'Jack and the Beanstalk', ...,
       "Madea's Family Reunion", 'Man About Town', 'Man of the Year'],
      dtype=object)

In [422]:
user_item_matrix__train = user_item_matrix.loc[Title]
user_item_matrix__train = user_item_matrix__train.astype(np.float64)
user_item_matrix__train

,Michael Cristofer,Mike Nichols,Chazz Palminteri,Norberto Barba,Eric Schaeffer,Steve Oedekerk,Charles Herman-Wurmfeld,John Wayne,2 directors,Thomas McGuane,...,Tom Moore,Blaine Novak,Chuck Sheetz,Glenn Tryon,Ken Weiderhorn,Frank Whaley,Jean de Limur,Roland D. Reed,Tod Browning,William Corrigan
Love by the Light of the Moon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Terrible Teddy, the Grizzly King",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jack and the Beanstalk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alice in Wonderland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Great Train Robbery,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lucky Number Slevin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mad Cowgirl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Madea's Family Reunion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Man About Town,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [423]:
%%time
U, S, VT = np.linalg.svd(user_item_matrix__train.T)
V = VT.T


CPU times: user 8min 13s, sys: 1min 27s, total: 9min 40s
Wall time: 1min 25s


In [424]:
U.shape

(3956, 3956)

In [425]:
V.shape


(14322, 14322)

In [426]:
S.shape

(3956,)

In [427]:
Sigma = np.diag(S)
Sigma.shape


(3956, 3956)

In [428]:
Sigma

array([[1.29517640e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.73240565e+02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.65374084e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.62695041e-15, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.54878407e-15, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.81710576e-27]])

In [429]:
r=3
Ur = U[:, :r]
Sr = Sigma[:r, :r]
Vr = V[:, :r]
# Матрица соотношения между новым дегустатором и латентными факторами
test_user = np.mat(user_item_matrix__test.values)
test_user = test_user.astype(np.float64)
test_user.shape, test_user

((1, 3956), matrix([[0., 0., 0., ..., 0., 0., 0.]]))

In [430]:
tmp = test_user * Ur * np.linalg.inv(Sr)
tmp

matrix([[-6.17532236e-02,  1.16101444e-05, -4.48165637e-20]])

In [431]:
test_user_result = np.array([tmp[0,0], tmp[0,1], tmp[0,2]])
test_user_result

array([-6.17532236e-02,  1.16101444e-05, -4.48165637e-20])

In [432]:
cos_sim = cosine_similarity(Vr, test_user_result.reshape(1, -1))
cos_sim[:10]

array([[ 1.00000000e+00],
       [ 1.00000000e+00],
       [-1.86493778e-04],
       [ 6.69110329e-04],
       [-9.99997151e-01],
       [ 1.00000000e+00],
       [-1.71061511e-18],
       [-7.08333726e-19],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])

In [433]:
cos_sim_list = cos_sim.reshape(-1, cos_sim.shape[0])[0]
cos_sim_list[:10]

array([ 1.00000000e+00,  1.00000000e+00, -1.86493778e-04,  6.69110329e-04,
       -9.99997151e-01,  1.00000000e+00, -1.71061511e-18, -7.08333726e-19,
        0.00000000e+00,  0.00000000e+00])

In [434]:
recommended_user_id = np.argsort(-cos_sim_list)[0]
recommended_user_id

12561

In [435]:
test_user

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [549]:
movieId_list = list(user_item_matrix.columns)
def film_name_by_movieid(ind):
    try:
        movieId = movieId_list[ind]
        flt_links = data2[data2['Director'] == movieId]
        tmdbId = int(flt_links['point'].values[0])
        md_links = data2[data2['id'] == tmdbId]
        res = md_links['title'].values[0]
        return res
    except:
        return '' 

In [551]:
i=1
for idx, item in enumerate(np.ndarray.flatten(np.array(test_user))):
    if item > 0:
        film_title = film_name_by_movieid(idx)
        print('{} - {} - {}'.format(idx, Title[i], item))
        if i==10000:
            break
        else:
            i+=1

3794 - Terrible Teddy, the Grizzly King - 80.0
